# Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://ag2ai.github.io/ag2/docs/Use-Cases/agent_chat).

This notebook is modified based on https://github.com/ag2ai/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://ag2ai.github.io/ag2/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo"],
    },
)

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

## Construct Agents

In [2]:
from pydantic import BaseModel


class ResponseModel(BaseModel):
    question: str
    short_answer: str
    reasoning: str
    difficulty: float


llm_config = {"config_list": config_list, "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
    response_format=ResponseModel,
)

## Start Chat

In [3]:
user_proxy.initiate_chat(assistant, message="What's up", max_turns=1, summary_method="last_msg")

User_proxy (to Assistant):

What's up

--------------------------------------------------------------------------------


PicklingError: Can't pickle <class 'openai.types.chat.parsed_chat_completion.ParsedChatCompletion[ResponseModel]'>: attribute lookup ParsedChatCompletion[ResponseModel] on openai.types.chat.parsed_chat_completion failed